In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import EfficientNetB4 
from keras_tuner import RandomSearch

# 데이터셋 경로 설정
train_dir = r'E:\AI\dataset_skeleton_sep\face\BicycleCrunch\training'
val_dir = r'E:\AI\dataset_skeleton_sep\face\BicycleCrunch\validation'
test_dir = r'E:\AI\dataset_skeleton_sep\face\BicycleCrunch\test'

# ImageDataGenerator 초기화
datagen = ImageDataGenerator(rescale=1./255)

# 훈련, 검증, 테스트 데이터셋을 위한 제너레이터 생성
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    shuffle=True)

validation_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    shuffle=False)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    shuffle=False)


Found 7874 images belonging to 8 classes.
Found 1691 images belonging to 8 classes.
Found 1687 images belonging to 8 classes.


In [2]:

base_model = EfficientNetB4(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False
#for layer in base_model.layers[-20:]:
#    layer.trainable = True

model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.BatchNormalization())
model.add(layers.Dense(8, activation = 'softmax'))  



In [3]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

model.compile(optimizer = optimizers.Adam(learning_rate = 0.005),  
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // train_generator.batch_size,
                    validation_data=validation_generator, 
                    validation_steps=validation_generator.samples // validation_generator.batch_size,
                    epochs = 25, 
                    verbose=1, 
                    callbacks=[early_stopping])

test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print('\n테스트 정확도:', test_acc)

Epoch 1/25
492/492 [==============================] - 46s 75ms/step - loss: 2.8592 - accuracy: 0.1255 - val_loss: 2.1194 - val_accuracy: 0.1256
Epoch 2/25
492/492 [==============================] - 35s 71ms/step - loss: 2.4250 - accuracy: 0.1326 - val_loss: 2.0826 - val_accuracy: 0.1286
Epoch 3/25
492/492 [==============================] - 35s 71ms/step - loss: 2.2356 - accuracy: 0.1270 - val_loss: 2.0861 - val_accuracy: 0.1232
Epoch 4/25
492/492 [==============================] - 35s 71ms/step - loss: 2.1432 - accuracy: 0.1220 - val_loss: 2.0810 - val_accuracy: 0.1280
Epoch 5/25
160/492 [========>.....................] - ETA: 19s - loss: 2.1031 - accuracy: 0.1190

KeyboardInterrupt: 